# YouTube Trending Video Dataset Cleaning

This notebook contains code to clean up [YouTube Trending Video Dataset from Kaggle](https://www.kaggle.com/rsrishav/youtube-trending-video-dataset).

The rules by which data cleaning is done depend very much on the task, so mine may be very different from yours.

My rules/stages:
- Combine datasets for all countries into one large pandas DataFrame
- Replace NaN in description with space (videos with no description)
- Delete all rows with missing values if any
- Keep records only with unique video IDs with the maximum number of views (the latest request)
- Delete entries where comments_disabled=True or ratings_disabled=True (where comments or information about likes/dislikes are unavailable). Note that all videos have dislikes unavailable from December 13, 2021
- Split tags with '|' and convert a list of strings into one string
- Delete non-ASCII and non-English characters from video title, channel title, description, and tags
- Delete rows where the video title or channel title doesn't contains English letters anymore
- Create a new comments column with " " (space for all rows)
- Rename columns to match snake case
- Delete next columns: ['categoryId', 'trending_date', 'thumbnail_link', 'comments_disabled', 'ratings_disabled']
- Reindex columns
- Reset index

This code allows you to get 136k rows dataset.

# Imports

In [1]:
import glob
import pandas as pd

# Combine datasets from all countries

In [2]:
# list of paths to all .csv files
datasets_filenames = glob.glob("data/*.csv")

In [3]:
def combine_datasets(filenames):
    """ Concatenete all dataframes from 'filenames' and reset index """
    
    list_of_df = []

    for filename in filenames:
        current_df = pd.read_csv(filename,
                                 parse_dates=['publishedAt', 'trending_date'])
        list_of_df.append(current_df)

    all_df = pd.concat(list_of_df)
    all_df.reset_index(drop=True, inplace=True)
        
    return all_df

In [4]:
combined_df = combine_datasets(datasets_filenames)
combined_df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11 22:21:49+00:00,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,22,2020-08-12 00:00:00+00:00,pietro|guedes|ingrid|ohara|pingrid|vlog|amigos...,263835,85095,487,4500,https://i.ytimg.com/vi/s9FH4rDMvds/default.jpg,False,False,"Salve rapaziada, neste vídeo me declarei pra e..."
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11 15:00:13+00:00,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,10,2020-08-12 00:00:00+00:00,JYP Entertainment|JYP|ITZY|있지|ITZY Video|ITZY ...,6000070,714310,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10 14:59:00+00:00,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,22,2020-08-12 00:00:00+00:00,OH JULIANA PARÓDIA|MC Niack PARÓDIA|PARÓDIAS|A...,2296748,39761,5484,0,https://i.ytimg.com/vi/3EfkCrXKZNs/default.jpg,True,False,Se inscrevam meus amores! 📬 Quer nos mandar al...
3,gBjox7vn3-g,Contos de Runeterra: Targon | A Estrada Tortuosa,2020-08-11 15:00:09+00:00,UC6Xqz2pm50gDCORYztqhDpg,League of Legends BR,20,2020-08-12 00:00:00+00:00,Riot|Riot Games|League of Legends|lol|trailer|...,300510,46222,242,2748,https://i.ytimg.com/vi/gBjox7vn3-g/default.jpg,False,False,Você se unirá aos Lunari e aos Solari em Targo...
4,npoUGx7UW7o,Entrevista com Thammy Miranda | The Noite (10/...,2020-08-11 20:04:02+00:00,UCEWOoncsrmirqnFqxer9lmA,The Noite com Danilo Gentili,23,2020-08-12 00:00:00+00:00,The Noite|The Noite com Danilo Gentili|Danilo ...,327235,22059,3972,2751,https://i.ytimg.com/vi/npoUGx7UW7o/default.jpg,False,False,Danilo Gentili recebe Thammy Miranda. Após pas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273175,_R0tSt5tTUU,A Grueling Footrace To An Abandoned Ghost Town!,2023-06-11 20:00:11+00:00,UCEjBDKfrqQI4TgzT9YLNT8g,Ghost Town Living,22,2023-06-18 00:00:00+00:00,Ghost Town Living|Cerro Gordo|Footrace|Death V...,246175,23217,0,1227,https://i.ytimg.com/vi/_R0tSt5tTUU/default.jpg,False,False,Check out https://drinklmnt.com/Brent for a FR...
2273176,wq1FgZy1LzE,Game Theory: Garten Of BanBan Lore Is... Somet...,2023-06-10 18:05:36+00:00,UCo_IB5145EVNcf8hw1Kku7w,The Game Theorists,20,2023-06-18 00:00:00+00:00,garten of banban|garten|banban|jumbo josh|gart...,2264185,108041,0,5348,https://i.ytimg.com/vi/wq1FgZy1LzE/default.jpg,False,False,Game Theory Is Now On Spotify!Check Out Some O...
2273177,uvebNBKOSTg,"bye, from Dream.",2023-06-09 21:40:08+00:00,UCTkXRDQl0luXxVQrRQvWS6w,Dream,20,2023-06-18 00:00:00+00:00,minecraft|dream|dream minecraft,6152817,459258,0,92458,https://i.ytimg.com/vi/uvebNBKOSTg/default.jpg,False,False,"bye, from Dream. I deleted my face reveal, and..."
2273178,LScLTYUTXAM,Brandon Crawford's Scoreless Inning | Pitching...,2023-06-11 23:54:18+00:00,UCpXMHgjrpnynDSV5mXpqImw,San Francisco Giants,17,2023-06-18 00:00:00+00:00,San Francisco Giants|SF Giants|SFGiants|Oracle...,138893,2662,0,237,https://i.ytimg.com/vi/LScLTYUTXAM/default.jpg,False,False,"For the first time in his Major League career,..."


In [5]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2273180 entries, 0 to 2273179
Data columns (total 16 columns):
 #   Column             Dtype              
---  ------             -----              
 0   video_id           object             
 1   title              object             
 2   publishedAt        datetime64[ns, UTC]
 3   channelId          object             
 4   channelTitle       object             
 5   categoryId         int64              
 6   trending_date      datetime64[ns, UTC]
 7   tags               object             
 8   view_count         int64              
 9   likes              int64              
 10  dislikes           int64              
 11  comment_count      int64              
 12  thumbnail_link     object             
 13  comments_disabled  bool               
 14  ratings_disabled   bool               
 15  description        object             
dtypes: bool(2), datetime64[ns, UTC](2), int64(5), object(7)
memory usage: 247.1+ MB


In [6]:
# dataset size - 1,074,418 rows, 
# but it contains only 199,728 unique video IDs
# data for December 13, 2021
len(combined_df['video_id'].unique())

398811

# Sample Dataset

In [7]:
sample_df = pd.read_csv("data/US_youtube_trending_data.csv", parse_dates=['publishedAt', 'trending_date'])
sample_df.reset_index(drop=True, inplace=True)

sample_df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12 00:00:00+00:00,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11 17:00:10+00:00,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12 00:00:00+00:00,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11 16:34:06+00:00,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12 00:00:00+00:00,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11 16:38:55+00:00,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12 00:00:00+00:00,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11 15:10:05+00:00,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12 00:00:00+00:00,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208983,_R0tSt5tTUU,A Grueling Footrace To An Abandoned Ghost Town!,2023-06-11 20:00:11+00:00,UCEjBDKfrqQI4TgzT9YLNT8g,Ghost Town Living,22,2023-06-18 00:00:00+00:00,Ghost Town Living|Cerro Gordo|Footrace|Death V...,246175,23217,0,1227,https://i.ytimg.com/vi/_R0tSt5tTUU/default.jpg,False,False,Check out https://drinklmnt.com/Brent for a FR...
208984,wq1FgZy1LzE,Game Theory: Garten Of BanBan Lore Is... Somet...,2023-06-10 18:05:36+00:00,UCo_IB5145EVNcf8hw1Kku7w,The Game Theorists,20,2023-06-18 00:00:00+00:00,garten of banban|garten|banban|jumbo josh|gart...,2264185,108041,0,5348,https://i.ytimg.com/vi/wq1FgZy1LzE/default.jpg,False,False,Game Theory Is Now On Spotify!Check Out Some O...
208985,uvebNBKOSTg,"bye, from Dream.",2023-06-09 21:40:08+00:00,UCTkXRDQl0luXxVQrRQvWS6w,Dream,20,2023-06-18 00:00:00+00:00,minecraft|dream|dream minecraft,6152817,459258,0,92458,https://i.ytimg.com/vi/uvebNBKOSTg/default.jpg,False,False,"bye, from Dream. I deleted my face reveal, and..."
208986,LScLTYUTXAM,Brandon Crawford's Scoreless Inning | Pitching...,2023-06-11 23:54:18+00:00,UCpXMHgjrpnynDSV5mXpqImw,San Francisco Giants,17,2023-06-18 00:00:00+00:00,San Francisco Giants|SF Giants|SFGiants|Oracle...,138893,2662,0,237,https://i.ytimg.com/vi/LScLTYUTXAM/default.jpg,False,False,"For the first time in his Major League career,..."


# Clean dataset

In [8]:
def clean_kaggle_dataset(dataset):
    """ 
    Clean YouTube Kaggle dataset:
        - Replace NaN in description with space
        - Delete missing values
        - Keep records only with unique video IDs 
          with the maximum number of views (the latest request)
        - Delete entries where comments_disabled=True
        - Convert "tags" field from list of strings to string
        - Delete non-ASCII characters and non-English from text columns
        - Rename columns to snake case, reorder add empty 'comments' column
        - Delete non-relevant columns
        - Reset index
    """

    clean_df = dataset.copy(deep=True)

    # Replace NaN in description with space
    clean_df["description"].fillna(" ", inplace=True)
    # Delete all rows with a missing values if any
    clean_df.dropna(inplace=True)

    # primary_key - unique 'video_id' with the largest number of views
    primary_key = clean_df.groupby("video_id")["view_count"].idxmax()
    # keep only most relevant records
    clean_df = clean_df.loc[primary_key]

    # delete rows with comments_disabled=True or ratings_disabled=True
    clean_df = clean_df[(clean_df['comments_disabled'] == False) &
                        (clean_df['ratings_disabled'] == False)]

    # Replace [None] in tags with space
    clean_df.loc[clean_df['tags'] == '[None]', 'tags'] = ' '
    # split tags with '|' and convert list to one string
    clean_df['tags'] = [' '.join(tag)
                        for tag in clean_df['tags'].str.split('|')]

    printable = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n'
    # delete non-ASCII and non-English characters
    for text_column in ['title', 'channelTitle', 'description', 'tags']:
        # for all rows in the column apply a filter
        # that only leaves characters from 'printable'
        # since filter does not return a string, then you need to use the join method
        clean_df[text_column] = clean_df[text_column].apply(
            lambda x: ''.join(filter(lambda xi: xi in printable, x)))

    # if there is not a single letter left
    # in the title of the video or in the channel title
    # the video is definitely not in English
    symbols = [c for c in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"]
    clean_df = clean_df[clean_df['title'].str.contains('|'.join(symbols))]
    clean_df = clean_df[clean_df['channelTitle'].str.contains(
        '|'.join(symbols))]

    # create new empty column
    clean_df['comments'] = " "

    # rename column to match snake case
    clean_df.rename(columns={'channelTitle': 'channel_title',
                             'publishedAt': 'published_at',
                             'channelId': 'channel_id'}, inplace=True)

    # delete non-relevant columns
    clean_df.drop(['categoryId', 'trending_date',
                   'thumbnail_link', 'comments_disabled',
                   'ratings_disabled'], axis=1, inplace=True)

    clean_df = clean_df.reindex(columns=['video_id', 'title', 'channel_id', 'channel_title',
                                         'published_at', 'view_count', 'likes', 'dislikes',
                                         'comment_count', 'tags', 'description', 'comments'])

    clean_df.reset_index(drop=True, inplace=True)

    return clean_df

In [9]:
clean_df = clean_kaggle_dataset(combined_df)
clean_df

,video_id,title,channel_id,channel_title,published_at,view_count,likes,dislikes,comment_count,tags,description,comments
0,---Pt7XVuOk,CH REVELAO MENINO OU MENINA? *Quarta Gravidez*,UC3PGilZDRq6wzd7mURXET2g,Sara Kellen Me de Trs,2022-12-24 20:42:26+00:00,520256,41456,0,2303,sarakellen medetrs vidanaroa youtuberdaroa vid...,Oi minhas lindas e meus lindos.Espero que gost...,
1,--0bCF-iK2E,Jadon Sancho Magical Skills & Goals,UC6UL29enLNe4mqwTfAyeNuw,Bundesliga,2021-07-01 10:00:00+00:00,433340,11276,110,1083,football soccer ftbol alemn Bundesliga season ...,Enjoy the best skills and goals from Jadon San...,
2,--14w5SOEUs,Migos - Avalanche (Official Video),UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-10 16:00:00+00:00,8034045,281817,4675,17321,Migos Avalanche Quality Control Music/Motown R...,Watch the the official video for Migos - Avala...,
3,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,UCXJEvxZSozjAAqhbMfhIArA,jf.okay,2022-03-09 23:19:08+00:00,565436,17771,0,1558,,,
4,--2Us2lFR8Y,JE CONSTRUIS UNE PICE SECRTE DANS MA MAISON,UCgl_xdd0kH27vMIZnE-_17w,ALEKS,2021-02-11 17:02:05+00:00,705147,94116,1105,4666,maison secret pice secrte secrte constuire je ...,Aprs avoir ragis PLEIN de pices SECRTES chez ...,
...,...,...,...,...,...,...,...,...,...,...,...,...
284432,zzwRrWlq1Jo,OS MELHORES PROFESSORES DO MUNDO,UCV306eHqgo0LvBf3Mh36AHg,Felipe Neto,2022-04-29 13:00:37+00:00,877977,110452,0,1771,felipe neto irmos neto professor colegio aluno...,LOJA DO CANAL - https://www.lolja.com.br/felip...,
284433,zzwv0MOoLPk,"If I Say This Word, I Reset My Speedrun",UCG6zBb8GZKo1XZW4eHdg-0Q,PointCrow,2023-02-09 20:19:44+00:00,249924,15054,0,392,elden ring eldenring the elden ring pointcrow ...,I attempt to speedrun the first boss in Elden ...,
284434,zzxPZwaA-8w,Gareth Bale brace secures dramatic comeback on...,UCEg25rdRZXg32iwai6N6l0w,Tottenham Hotspur,2021-05-23 21:00:31+00:00,2144129,32744,823,2037,Spurs Tottenham Hotspur Tottenham Leicester ...,Two minute highlights from Tottenham Hotspur's...,
284435,zzyTrU2IJsQ,Sarkaru Vaari Paata Movie Review | Mahesh Babu...,UC-qTldS8DNIGOfIVc0G8t-w,THYVIEW,2022-05-12 01:58:39+00:00,428641,27820,0,967,Super Star Mahesh Babu Keerthy Suresh Vennela ...,ello All welcome to thyview. here is our non-s...,


In [10]:
# check that group by worked correctly
len(clean_df['video_id'].unique())

284437

In [11]:
# save dataset
clean_df.to_csv('youtube_kaggle_clean_dataset.tsv', sep='\t', index=False, header=True)

In [12]:
clean_sample = clean_kaggle_dataset(sample_df)
clean_sample

,video_id,title,channel_id,channel_title,published_at,view_count,likes,dislikes,comment_count,tags,description,comments
0,--14w5SOEUs,Migos - Avalanche (Official Video),UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-10 16:00:00+00:00,6823249,262692,4107,16445,Migos Avalanche Quality Control Music/Motown R...,Watch the the official video for Migos - Avala...,
1,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,UCXJEvxZSozjAAqhbMfhIArA,jf.okay,2022-03-09 23:19:08+00:00,538485,17290,0,1439,,,
2,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,UClBKH8yZRcM4AsRjDVEdjMg,Television Academy,2021-09-20 01:03:32+00:00,682609,8029,369,723,,Hannah Waddingham wins the Emmy for Supporting...,
3,--5-brQiQFg,Washington Commanders vs. San Francisco 49ers ...,UCDVYQ4Zhbm3S2dlz7P1GBDg,NFL,2022-12-25 00:30:17+00:00,1280997,14603,0,2078,,Check out our other channels:NFL Mundo https:/...,
4,--DKkzWVh-E,Why Retaining Walls Collapse,UCMOqf8ab-42UUQIdVoKwjlQ,Practical Engineering,2021-12-07 13:00:00+00:00,623949,29991,320,998,retaining wall New Jersey highway Direct Conne...,One of the most important (and innocuous) part...,
...,...,...,...,...,...,...,...,...,...,...,...,...
37132,zzCrFWjKPy8,Season 1 Episode 3 Preview | House of the Drag...,UCQzdMyuz0Lf4zo4uGcEujFw,GameofThrones,2022-08-29 02:00:10+00:00,2603821,47431,0,3586,,New episodes premiere Sundays at 9 pm ET on HB...,
37133,zzd4ydafGR0,Lil Tjay - Calling My Phone (feat. 6LACK) [Off...,UCEB4a5o_6KfjxHwNMnmj54Q,Lil Tjay,2021-02-12 05:03:49+00:00,13974461,728439,7579,57012,Lil Tjay Steady Calling My Phone Calling My Ph...,Official audio for Calling My Phone by Lil Tja...,
37134,zziBybeSAtw,PELICANS at LAKERS | FULL GAME HIGHLIGHTS | Ja...,UCWJ2lWNubArHWmf3FIHbfcQ,NBA,2021-01-16 05:39:05+00:00,2598512,20024,989,2872,NBA G League Basketball game-0022000187 Lakers...,PELICANS at LAKERS | FULL GAME HIGHLIGHTS | Ja...,
37135,zzk09ESX7e0,[MV] (MAMAMOO) - Where Are We Now,UCuhAUMLzJxlP1W7mEk0_6lA,MAMAMOO,2021-06-02 09:00:10+00:00,9389223,584431,3403,77267,MAMAMOO WAW WAW MAMAMOO WAW Where Are We Now...,[MV] (MAMAMOO) - Where Are We NowInstagram: h...,


In [13]:
clean_sample.to_csv('youtube_kaggle_sample_dataset.tsv', sep='\t', index=False, header=True)